In [0]:
from keras import optimizers
from keras.models import Sequential,Model
from keras.layers import Dense,CuDNNLSTM,Bidirectional
from keras.layers import Activation,Dropout,Flatten
import keras.optimizers as optimizer
import keras
import sys

In [2]:
from google.colab import files
files.upload()

Saving Kanye.txt to Kanye.txt


{'Kanye.txt': b'\r\nDrug dealin\' just to get by\r\nStack ya\' money \'til it get sky high\r\nWe wasn\'t s\'posed to make it past 25\r\nJoke\'s on you, we still alive\r\nThrow your hands up in the sky and say:\r\n""We don\'t care what people say""\r\n\r\n[ 1]\r\nIf this is your first time hearin\' this\r\nYou are about to experience somethin\' so cold, man\r\nWe never had nothin\' handed, took nothin\' for granted\r\nTook nothin\' from no man, man, I\'m my own man\r\nBut as a shorty I looked up to the dopeman\r\nOnly adult man I knew that wasn\'t broke, man\r\nFlickin\' Starter coats, man\r\nMan, you don\'t know, man\r\nWe don\'t care what people say\r\nThis is for my niggas outside all winter\r\n\'Cause this summer they ain\'t finna say: ""next summer I\'m finna""\r\nSittin\' in the hood like community colleges\r\nThis dope money here is Lil\' Trey\'s scholarship\r\n\'Cause ain\'t no tuition for having no ambition\r\nAnd ain\'t no loans for sittin\' your ass at home\r\nSo we forced to

In [0]:
temp = open('Kanye.txt',encoding = 'UTF-8').read()

In [4]:
temp[0:250]

'\nDrug dealin\' just to get by\nStack ya\' money \'til it get sky high\nWe wasn\'t s\'posed to make it past 25\nJoke\'s on you, we still alive\nThrow your hands up in the sky and say:\n""We don\'t care what people say""\n\n[ 1]\nIf this is your first time hearin\' th'

In [0]:
temp = temp.lower()
# $ =  Enter
temp = temp.replace('\n','$')

In [0]:
lyrics = []
for i,x in enumerate(temp):
  if x == '[':
    while(True):
      if temp[i].strip() != ']':
        i = i+1
      else:
        break
  elif x.isdigit() or (x not in['$','?','.',' ',' \ '] and not x.isalpha()):
    pass
  else:
    lyrics.append(x)

In [0]:
chars = sorted(list(set(lyrics)))

In [8]:
len(chars)

30

In [9]:
str(chars)

"[' ', '$', '.', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']"

In [0]:
intToChars = dict((i, c) for i, c in enumerate(chars))
charsToInt = dict((i, c) for c, i in enumerate(chars))

In [0]:
batch = 20
vocab_size = len(chars)
lyrics_size = len(lyrics)

In [0]:
X = []
y = []
for i in range(0,lyrics_size - batch - 1):
  tempX = lyrics[i:(i+batch)]
  tempY = lyrics[i+batch]
  
  X.append([charsToInt[ch] for ch in tempX])
  y.append(charsToInt[tempY])

In [13]:
len(y)

304573

In [0]:
import numpy as np
X = np.reshape(X,(len(y),batch,1))
y = np.reshape(y,(len(y),1))

In [0]:
X = X/30.0

In [0]:
labels = keras.utils.to_categorical(y, num_classes=30, dtype='float32')

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.10)

In [0]:
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint
model = Sequential()

# add LSTM layer
model.add(Bidirectional(CuDNNLSTM(128, input_shape=(batch, 1))))

# add Softmax layer to output one character 
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

# compile the model and pick the loss and optimizer
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=5, min_lr=0.000001)

filepath="weights-improvement-{epoch:02d}-{acc:.2f}.h5"

checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
callbacks_list = [reduce_lr,checkpoint]

model.compile(loss='categorical_crossentropy', metrics = ['accuracy'], optimizer='RMSprop')


In [20]:
# train the model
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1,
                              patience=5, min_lr=0.000000001)

filepath="weights-improvement-{epoch:02d}-{acc:.2f}.h5"

checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
callbacks_list = [reduce_lr,checkpoint]

model.fit(X_train, y_train, batch_size=128, epochs=100,validation_data = (X_test,y_test),verbose = 1,callbacks = callbacks_list)

Train on 274115 samples, validate on 30458 samples
Epoch 1/100
274115/274115 [==============================] - 33s 119us/step - loss: 2.7381 - acc: 0.2265 - val_loss: 2.6710 - val_acc: 0.2426
Epoch 2/100
274115/274115 [==============================] - 32s 117us/step - loss: 2.6034 - acc: 0.2578 - val_loss: 2.5043 - val_acc: 0.2827
Epoch 3/100
274115/274115 [==============================] - 32s 116us/step - loss: 2.4141 - acc: 0.3003 - val_loss: 2.3120 - val_acc: 0.3255
Epoch 4/100
274115/274115 [==============================] - 32s 117us/step - loss: 2.2545 - acc: 0.3392 - val_loss: 2.1479 - val_acc: 0.3686
Epoch 5/100
274115/274115 [==============================] - 32s 117us/step - loss: 2.1403 - acc: 0.3687 - val_loss: 2.1470 - val_acc: 0.3650
Epoch 6/100
274115/274115 [==============================] - 32s 116us/step - loss: 2.0525 - acc: 0.3899 - val_loss: 2.1234 - val_acc: 0.3722
Epoch 7/100
274115/274115 [==============================] - 32s 116us/step - loss: 1.9833 - acc:

In [0]:
model.save_weights('my_model_weights.h5')
model.save('my_model.h5')

files.download('my_model_weights.h5')
files.download('my_model.h5')

In [25]:
input_data = 'oh my god this fame '
#input_data = 'waiting patiently i '

print(input_data)
for i in range(400):  
  temp = [charsToInt[ch] for ch in input_data]
  temp = np.asarray(temp)
  temp = temp.reshape((1,20,1))
  temp = temp/30 
  hum = model.predict(temp)
  hum = np.argmax(hum)
  ch = intToChars[hum]
  if ch != '$':
    sys.stdout.write(ch)
  else:
    print('')
  input_data = input_data[1:]
  input_data = input_data + ch

oh my god this fame 
in the alright sweats of you gon stupod on mass cant let me grees what i told hard if i could go for the be they want to in the sky and say
we dont care what ment be wears take it the dont spok hollow

hook kanye west  jayz
this is in not just down on my nama than shit cont trying to cant set mothing good to my fault in a joes you will a pown
and remember the charme the soul
and they say and they 